# Introdução a sistemas de recomendação

In [1]:
import pandas as pd

## Filmes dataset

In [2]:
filmes = pd.read_csv(r"C:\Users\galgu\Documents\araugus\Studies\Machine Learning\Bases\ml-latest\movies.csv")
filmes = filmes.set_index('movieId')
filmes.head()

,title,genres
movieId,,
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,Jumanji (1995),Adventure|Children|Fantasy
3,Grumpier Old Men (1995),Comedy|Romance
4,Waiting to Exhale (1995),Comedy|Drama|Romance
5,Father of the Bride Part II (1995),Comedy


## Notas dataset

In [3]:
notas = pd.read_csv(r"C:\Users\galgu\Documents\araugus\Studies\Machine Learning\Bases\ml-latest\ratings.csv")
notas.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,1225734739
1,1,110,4.0,1225865086
2,1,158,4.0,1225733503
3,1,260,4.5,1225735204
4,1,356,5.0,1225735119


In [4]:
notas.describe().round(2)

,userId,movieId,rating,timestamp
count,33832162.00,33832162.00,33832162.00,3.383216e+07
mean,165437.98,28313.48,3.54,1.269362e+09
std,95341.22,49928.65,1.06,2.541023e+08
min,1.00,1.00,0.50,7.896520e+08
25%,82953.00,1219.00,3.00,1.046718e+09
50%,166129.00,3263.00,4.00,1.264740e+09
75%,247450.00,40491.00,4.00,1.496919e+09
max,330975.00,288983.00,5.00,1.689843e+09


## Primeira tentativa de Recomendação - Mais reviews

In [5]:
total_de_votos = notas['movieId'].value_counts()
total_de_votos.head()

movieId
318     122296
356     113581
296     108756
2571    107056
593     101802
Name: count, dtype: int64

In [6]:
filmes.loc[318]

title     Shawshank Redemption, The (1994)
genres                         Crime|Drama
Name: 318, dtype: object

In [7]:
filmes['total_de_votos']=total_de_votos
filmes.head()

,title,genres,total_de_votos
movieId,,,
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,76813.0
2,Jumanji (1995),Adventure|Children|Fantasy,30209.0
3,Grumpier Old Men (1995),Comedy|Romance,15820.0
4,Waiting to Exhale (1995),Comedy|Drama|Romance,3028.0
5,Father of the Bride Part II (1995),Comedy,15801.0


In [8]:
filmes.sort_values('total_de_votos', ascending=False).head(10)

,title,genres,total_de_votos
movieId,,,
318,"Shawshank Redemption, The (1994)",Crime|Drama,122296.0
356,Forrest Gump (1994),Comedy|Drama|Romance|War,113581.0
296,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,108756.0
2571,"Matrix, The (1999)",Action|Sci-Fi|Thriller,107056.0
593,"Silence of the Lambs, The (1991)",Crime|Horror|Thriller,101802.0
260,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Sci-Fi,97202.0
2959,Fight Club (1999),Action|Crime|Drama|Thriller,86207.0
527,Schindler's List (1993),Drama|War,84232.0
480,Jurassic Park (1993),Action|Adventure|Sci-Fi|Thriller,83026.0


In [9]:
notas_medias = notas.groupby('movieId').mean()['rating']
notas_medias.head()

movieId
1    3.893508
2    3.278179
3    3.171271
4    2.868395
5    3.076957
Name: rating, dtype: float64

In [10]:
filmes['notas_medias'] = notas_medias
filmes.sort_values('total_de_votos', ascending=False).head(10)

,title,genres,total_de_votos,notas_medias
movieId,,,,
318,"Shawshank Redemption, The (1994)",Crime|Drama,122296.0,4.416792
356,Forrest Gump (1994),Comedy|Drama|Romance|War,113581.0,4.068189
296,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,108756.0,4.191778
2571,"Matrix, The (1999)",Action|Sci-Fi|Thriller,107056.0,4.160631
593,"Silence of the Lambs, The (1991)",Crime|Horror|Thriller,101802.0,4.150287
260,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Sci-Fi,97202.0,4.092400
2959,Fight Club (1999),Action|Crime|Drama|Thriller,86207.0,4.236019
527,Schindler's List (1993),Drama|War,84232.0,4.242337
480,Jurassic Park (1993),Action|Adventure|Sci-Fi|Thriller,83026.0,3.689013


## Segunda recomendação - Maior nota média + filtros

In [11]:
filmes.sort_values('notas_medias', ascending=False).head(10)

,title,genres,total_de_votos,notas_medias
movieId,,,,
267038,Christmas in the Rockies (2021),Romance,1.0,5.0
282157,A Royal Corgi Christmas (2022),Comedy|Romance,1.0,5.0
281624,Hidden (2020),Documentary,1.0,5.0
251222,İstanbul Beneath My Wings (1996),Adventure,1.0,5.0
246624,Two for the Win (2021),Romance,1.0,5.0
216789,Destination: Pluto Beyond the Flyby (2016),Documentary,1.0,5.0
200088,Little Loopers (2015),(no genres listed),1.0,5.0
200086,2BPerfectlyHonest (2004),(no genres listed),1.0,5.0
268808,A Ballad of Love (1970),Romance,1.0,5.0


In [12]:
filmes.query('total_de_votos >= 10').sort_values('notas_medias', ascending=False).head(10)

,title,genres,total_de_votos,notas_medias
movieId,,,,
247466,Money Heist: The Phenomenon (2020),Documentary,10.0,4.650000
102672,New York: A Documentary Film (1999),Documentary,11.0,4.500000
171011,Planet Earth II (2016),Documentary,2041.0,4.451739
206212,Evaru (2019),Crime|Thriller,10.0,4.450000
159817,Planet Earth (2006),Documentary,3015.0,4.448093
170705,Band of Brothers (2001),Action|Drama|War,2835.0,4.423986
279770,Kantara (2022),Action|Mystery|Thriller,13.0,4.423077
242210,James Acaster: Cold Lasagne Hate Myself 1999 (...,Comedy,19.0,4.421053
318,"Shawshank Redemption, The (1994)",Crime|Drama,122296.0,4.416792


In [13]:
filmes_com_mais_de_50_votos = filmes.query('total_de_votos > 50')
filmes_com_mais_de_50_votos.sort_values('notas_medias', ascending=False).head(10)

,title,genres,total_de_votos,notas_medias
movieId,,,,
171011,Planet Earth II (2016),Documentary,2041.0,4.451739
159817,Planet Earth (2006),Documentary,3015.0,4.448093
170705,Band of Brothers (2001),Action|Drama|War,2835.0,4.423986
318,"Shawshank Redemption, The (1994)",Crime|Drama,122296.0,4.416792
171495,Cosmos,(no genres listed),625.0,4.343200
202439,Parasite (2019),Comedy|Drama,12399.0,4.329946
858,"Godfather, The (1972)",Crime|Drama,75004.0,4.326603
179135,Blue Planet II (2017),Documentary,1267.0,4.312944
220528,Twelve Angry Men (1954),Drama,332.0,4.305723


## Terceira recomendação - Indicando filmes que gostei

In [14]:
eu_assisti = [1, 21, 19, 10, 11, 7, 2]
filmes.loc[eu_assisti]

,title,genres,total_de_votos,notas_medias
movieId,,,,
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,76813.0,3.893508
21,Get Shorty (1995),Comedy|Crime|Thriller,26183.0,3.565520
19,Ace Ventura: When Nature Calls (1995),Comedy,26642.0,2.668062
10,GoldenEye (1995),Action|Adventure|Thriller,34942.0,3.434835
11,"American President, The (1995)",Comedy|Drama|Romance,20256.0,3.658768
7,Sabrina (1995),Comedy|Romance,15596.0,3.372339
2,Jumanji (1995),Adventure|Children|Fantasy,30209.0,3.278179


In [15]:
filmes.query("genres=='Adventure|Children|Fantasy'")

,title,genres,total_de_votos,notas_medias
movieId,,,,
2,Jumanji (1995),Adventure|Children|Fantasy,30209.0,3.278179
60,"Indian in the Cupboard, The (1995)",Adventure|Children|Fantasy,7813.0,3.064892
126,"NeverEnding Story III, The (1994)",Adventure|Children|Fantasy,1590.0,2.336792
1009,Escape to Witch Mountain (1975),Adventure|Children|Fantasy,2862.0,3.139937
2043,Darby O'Gill and the Little People (1959),Adventure|Children|Fantasy,1217.0,3.281430
...,...,...,...,...
280890,Slumberland (2022),Adventure|Children|Fantasy,48.0,3.312500
282023,Jim Button and the Wild 13 (2020),Adventure|Children|Fantasy,2.0,3.250000
285269,The Magic Flute (2022),Adventure|Children|Fantasy,1.0,3.000000


In [16]:
aventura_infantil_e_fantasia = filmes_com_mais_de_50_votos.query("genres=='Adventure|Children|Fantasy'")
aventura_infantil_e_fantasia.sort_values('notas_medias', ascending=False).head(10)

,title,genres,total_de_votos,notas_medias
movieId,,,,
4896,Harry Potter and the Sorcerer's Stone (a.k.a. ...,Adventure|Children|Fantasy,36127.0,3.695159
2161,"NeverEnding Story, The (1984)",Adventure|Children|Fantasy,13617.0,3.517735
80748,Alice in Wonderland (1933),Adventure|Children|Fantasy,125.0,3.516000
141874,The Adventures of Buratino (1975),Adventure|Children|Fantasy,59.0,3.457627
50601,Bridge to Terabithia (2007),Adventure|Children|Fantasy,2697.0,3.435855
185231,"The Chronicles of Narnia: The Lion, the Witch ...",Adventure|Children|Fantasy,76.0,3.407895
41566,"Chronicles of Narnia: The Lion, the Witch and ...",Adventure|Children|Fantasy,17042.0,3.391738
82169,Chronicles of Narnia: The Voyage of the Dawn T...,Adventure|Children|Fantasy,2968.0,3.284367
2043,Darby O'Gill and the Little People (1959),Adventure|Children|Fantasy,1217.0,3.281430


In [17]:
aventura_infantil_e_fantasia.drop(eu_assisti, errors='ignore')\
                            .sort_values('notas_medias', ascending=False)\
                            .head(10)

,title,genres,total_de_votos,notas_medias
movieId,,,,
4896,Harry Potter and the Sorcerer's Stone (a.k.a. ...,Adventure|Children|Fantasy,36127.0,3.695159
2161,"NeverEnding Story, The (1984)",Adventure|Children|Fantasy,13617.0,3.517735
80748,Alice in Wonderland (1933),Adventure|Children|Fantasy,125.0,3.516000
141874,The Adventures of Buratino (1975),Adventure|Children|Fantasy,59.0,3.457627
50601,Bridge to Terabithia (2007),Adventure|Children|Fantasy,2697.0,3.435855
185231,"The Chronicles of Narnia: The Lion, the Witch ...",Adventure|Children|Fantasy,76.0,3.407895
41566,"Chronicles of Narnia: The Lion, the Witch and ...",Adventure|Children|Fantasy,17042.0,3.391738
82169,Chronicles of Narnia: The Voyage of the Dawn T...,Adventure|Children|Fantasy,2968.0,3.284367
2043,Darby O'Gill and the Little People (1959),Adventure|Children|Fantasy,1217.0,3.281430
